In [1]:
import numpy as np
import pickle
import random
import torch

from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.nn.utils import clip_grad_norm_ as clip_grad_norm
from torch.utils.data import DataLoader

In [2]:
seed = 1

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print(torch.randn(5))

tensor([ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519])


# Common functions

In [3]:
def train(model, trn_dataloader, optimizer, criterion, device=torch.device('cpu')):
    total_batch, correct_pred, total_sample = 0, 0, 0
    model.train()

    for batch, label in trn_dataloader:
        total_batch += 1

        batch = batch.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        output = model(batch)
        loss = criterion(output, label)
        loss.backward()
        clip_grad_norm(model.parameters(), 1.0)
        optimizer.step()
        trn_loss = loss.data.item()
        pred_label = output.argmax(dim=1)
        correct_pred += (pred_label == label).sum()
        total_sample += label.size()[0]

        if total_batch % 50 == 0:
            print(f"#{total_batch}: trn loss = {trn_loss:.4f} | trn acc = {(correct_pred/total_sample):.4f}")
        
    return model

In [4]:
def eval(model, dataloader, criterion, device=torch.device('cpu')):
    total_loss, total_batch = 0, 0
    correct_pred, total_sample = 0, 0
    model.eval()

    for batch, label in dataloader:
        batch = batch.to(device)
        label = label.to(device)

        output = model(batch)

        total_batch += 1
        loss = criterion(output, label)
        total_loss += loss.data.item()

        pred_label = output.argmax(dim=1)
        correct_pred += (pred_label == label).sum()
        total_sample += label.size()[0]

    avg_loss = total_loss / total_batch
    acc = correct_pred.item() / total_sample

    return avg_loss, acc

# Load data

In [5]:
y_trn = pickle.load(open("../preprocessed_embeddings/elmo_trn_title_labels.pkl", "rb"))
y_val = pickle.load(open("../preprocessed_embeddings/elmo_val_title_labels.pkl", "rb"))
y_tst = pickle.load(open("../preprocessed_embeddings/elmo_tst_title_labels.pkl", "rb"))

In [6]:
x_trn = pickle.load(open("../preprocessed_embeddings/elmo_trn_title.pkl", "rb")).tolist()
x_val = pickle.load(open("../preprocessed_embeddings/elmo_val_title.pkl", "rb")).tolist()
x_tst = pickle.load(open("../preprocessed_embeddings/elmo_tst_title.pkl", "rb")).tolist()

In [7]:
batch_size = 128

### Training set
trn_dataset = []
for i in range(len(x_trn)):
    trn_dataset.append((torch.tensor(x_trn[i]), y_trn[i]))

del x_trn
del y_trn
trn_dataloader = DataLoader(trn_dataset, batch_size)

### Validation set
val_dataset = []
for i in range(len(x_val)):
    val_dataset.append((torch.tensor(x_val[i]), y_val[i]))

del x_val
del y_val
val_dataloader = DataLoader(val_dataset, batch_size)

### Test set
tst_dataset = []
for i in range(len(x_tst)):
    tst_dataset.append((torch.tensor(x_tst[i]), y_tst[i]))

del x_tst
del y_tst
tst_dataloader = DataLoader(tst_dataset, batch_size)

# Convolutional Neural Network

In [28]:
class CNN(nn.Module):
    def __init__(self, drop_rate=0.0, kernel_size=4, embed_size=1024, class_size=2):
        super(CNN, self).__init__()

        # ---------------------------------
        # Configuration
        self.dropout = nn.Dropout(drop_rate)
        self.filter = nn.Conv1d(embed_size, embed_size, kernel_size)
        self.fc = nn.Linear(embed_size, class_size)

    def forward(self, batch):
        # Batch -> B x L x E
        x = batch.permute(0,2,1) # x -> B x E x L
        x = self.dropout(x)

        x = self.filter(x)
        x = torch.sigmoid(x)
        x = torch.max(x, dim=2)[0]

        out = self.fc(x)

        return out

In [29]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = CNN().to(device)

optimizer = optim.Adam(
    params=model.parameters(),
    lr=0.001
)

criterion = nn.CrossEntropyLoss()

In [30]:
for e in range(5):
    print(f'Epoch {e}\n') if e == 0 else print(f'\nEpoch {e}\n')
    model = train(model, trn_dataloader, optimizer, criterion, device)
    trn_loss, trn_acc = eval(model, trn_dataloader, criterion, device)
    val_loss, val_acc = eval(model, val_dataloader, criterion, device)
    print(f"\ntrn loss = {trn_loss} trn acc = {trn_acc} val loss = {val_loss} val acc = {val_acc}")

Epoch 0

#50: trn loss = 0.0012 | trn acc = 0.9720
#100: trn loss = 0.0005 | trn acc = 0.9852
#150: trn loss = 0.0022 | trn acc = 0.9882
#200: trn loss = 0.0304 | trn acc = 0.9906

trn loss = 0.05928844698463849 trn acc = 0.9781514232233912 val loss = 0.16929941649037314 val acc = 0.9428442335931323

Epoch 1

#50: trn loss = 0.0014 | trn acc = 0.9955
#100: trn loss = 0.0005 | trn acc = 0.9977
#150: trn loss = 0.0003 | trn acc = 0.9982
#200: trn loss = 0.0005 | trn acc = 0.9985

trn loss = 0.015971828409890573 trn acc = 0.9936745627057381 val loss = 0.0568739628901832 val acc = 0.9793290410030498

Epoch 2

#50: trn loss = 0.0000 | trn acc = 0.9991
#100: trn loss = 0.0001 | trn acc = 0.9995
#150: trn loss = 0.0000 | trn acc = 0.9996
#200: trn loss = 0.0000 | trn acc = 0.9997

trn loss = 0.011722615522003675 trn acc = 0.9951913767507907 val loss = 0.053237875625661346 val acc = 0.9830565909861064

Epoch 3

#50: trn loss = 0.0000 | trn acc = 0.9994
#100: trn loss = 0.0000 | trn acc = 0.999

In [31]:
val_loss, val_acc = eval(model, val_dataloader, criterion, device)
print(f"Validation Loss: {val_loss}")
print(f"Validation Acc: {val_acc}")

Validation Loss: 0.019983101046919468
Validation Acc: 0.9941262848751835


In [32]:
tst_loss, tst_acc = eval(model, tst_dataloader, criterion, device)
print(f"Test Loss: {tst_loss}")
print(f"Test Acc: {tst_acc}")

Test Loss: 0.038260751161850036
Test Acc: 0.9866757000903342
